figure out how to filter similar nba players by position

try to make good/great amplify the next word

In [1]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from num2words import num2words

In [23]:
ind_to_prospect = {}
prospect_to_ind = {}
prospects = []
with open("prospect_to_docs.json") as f:
    for l in f:
        line = json.loads(l)
        for ind, (prospect, text) in enumerate(line.items()):
            ind_to_prospect[ind] = prospect
            prospect_to_ind[prospect] = ind
            prospects.append(text)
with open("prospect_to_position.json") as f:
    for l in f:
        prospect_to_position = json.loads(l)
        
            
ind_to_player = {}
player_to_ind = {}
players = []
with open("curr_player_to_docs.json") as f:
    for l in f:
        line = json.loads(l)
        for ind, (player, text) in enumerate(line.items()):
            ind_to_player[ind] = player
            player_to_ind[player] = ind
            players.append(text)
with open("curr_player_to_position.json") as f:
    for l in f:
        player_to_position = json.loads(l)

In [13]:
def tokenize(text):
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    stems = []
    for tok in tokens:
        try:
            i = int(tok)
            stems.append(num2words(i))
        except ValueError:
            stems.append(stemmer.stem(tok))
    #stems = [stemmer.stem(token) for token in tokens]
    return stems
tfidf = TfidfVectorizer(stop_words="english", tokenizer=tokenize, ngram_range=(1, 3), norm=None)
docs = tfidf.fit(players+prospects)
prospect_docs = tfidf.transform(prospects).toarray()
player_docs = tfidf.transform(players).toarray()

In [4]:
def find_similarity(query, topk=10):
    transformed = tfidf.transform([query]).toarray().flatten()
    sims = []
    for ind, row in enumerate(prospect_docs):
        doc = row.flatten()
        if not np.all(doc == 0.0):
            dotted = np.dot(doc, transformed)
            sim = dotted/(np.linalg.norm(doc)*np.linalg.norm(transformed))
            sims.append((sim, ind_to_prospect[ind]))
    sorted_sims = sorted(sims, key=lambda x:x[0], reverse=True)
    for i in range(1, topk + 1):
        out = sorted_sims[i-1]
        print("Rank {}: {}, Similarity: {}".format(i, out[1], out[0]))

In [16]:
find_similarity("shot blocking center")

Rank 1: Jarrett Allen, Similarity: 0.0365620408138
Rank 2: Ike Anigbogu, Similarity: 0.0147625275128
Rank 3: Omer Yurtseven, Similarity: 0.0145358353667
Rank 4: Tacko Fall, Similarity: 0.0139476044011
Rank 5: Laurynas Birutis, Similarity: 0.0125170344411
Rank 6: Marques Bolden, Similarity: 0.00973460396599
Rank 7: Isaiah Hartenstein, Similarity: 0.00873534153633
Rank 8: Chimezie Metu, Similarity: 0.00831078864989
Rank 9: Jordan Bell, Similarity: 0.00814097562955
Rank 10: Thomas Bryant, Similarity: 0.00801542919298


In [19]:
def find_similar_players(prospect, topk=10):
    if prospect not in prospect_to_ind:
        print "Not a valid prospect"
        return
    prospect_position = prospect_to_position[prospect]
    prospect_ind = prospect_to_ind[prospect]
    prospect_doc = prospect_docs[prospect_ind]
    sims = []
    for ind, row in enumerate(player_docs):
        name = ind_to_player[ind]
        position = player_to_position[name]
        if position in prospect_position:
            doc = row.flatten()
            if not np.all(doc == 0.0):
                dotted = np.dot(doc, prospect_doc)
                sim = dotted/(np.linalg.norm(doc)*np.linalg.norm(prospect_doc))
                sims.append((sim, name))
    sorted_sims = sorted(sims, key=lambda x:x[0], reverse=True)
    for i in range(1, topk + 1):
        out = sorted_sims[i-1]
        print("Rank {}: {}, Similarity: {}".format(i, out[1], out[0]))

In [25]:
find_similar_players("Markelle Fultz")

Rank 1: Ricky Rubio, Similarity: 0.244465496014
Rank 2: Derrick Rose, Similarity: 0.224699966666
Rank 3: Brandon Jennings, Similarity: 0.220318984497
Rank 4: Sergio Rodriguez, Similarity: 0.21498066484
Rank 5: Greivis Vasquez, Similarity: 0.21166026024
Rank 6: Marcus Williams %28UConn%29, Similarity: 0.209337499833
Rank 7: Lorenzo Brown, Similarity: 0.207134659594
Rank 8: Kemba Walker, Similarity: 0.205172956385
Rank 9: Ty Lawson, Similarity: 0.204727886132
Rank 10: Darren Collison, Similarity: 0.203820472302
